# EmergencyZIP AI

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv


In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.8 MB/s eta 0:00:00


In [3]:
patient_symptom_df = pd.read_csv('/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv')
patient_symptom_df

,PatientID,ZipCode,Age,Gender,Symptoms
0,P001,70112,32,Female,Joint pain;Runny nose;Loss of taste;Sore throat
1,P002,15213,87,Female,Cough;Fever;Fatigue;Chest pain;Joint pain
2,P003,46201,21,Non-binary,Swelling;Blurred vision;Rash
3,P004,20001,46,Male,Chest tightness;Sneezing;Fever;Breathing diffi...
4,P005,96813,38,Non-binary,Vomiting;Headache;Runny nose;Chest pain;Breath...
...,...,...,...,...,...
95,P096,92101,71,Non-binary,Low-grade fever;Joint pain;Headache
96,P097,20001,80,Female,Vomiting;Abdominal cramps;Chest tightness;Abdo...
97,P098,85001,77,Female,Loss of smell;Abdominal pain
98,P099,84101,90,Non-binary,Fatigue;Blurred vision


In [4]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [5]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [6]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [7]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [8]:
patient_symptom_df.iloc[0].to_string()

'PatientID                                               P001\nZipCode                                                70112\nAge                                                       32\nGender                                                Female\nSymptoms     Joint pain;Runny nose;Loss of taste;Sore throat'

In [9]:
import typing_extensions as typing

class MedicalPatient(typing.TypedDict): 
    zipcode: int
    age: int
    gender: str
    symptoms: list[str]

def get_json_form(patient_symptom_entry): 
    entry_str = patient_symptom_entry.to_string()
    json_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=types.GenerateContentConfig(
            temperature=0.1, 
            response_mime_type="application/json", 
            response_schema=MedicalPatient
        ), 
        contents=entry_str
    )
    json_form = json_response.text
    return json_form

patient_json_form = get_json_form(patient_symptom_df.iloc[0])
print(patient_json_form)

{
  "zipcode": 70112,
  "age": 32,
  "gender": "Female",
  "symptoms": [
    "Joint pain",
    "Runny nose",
    "Loss of taste",
    "Sore throat"
  ]
}


In [10]:
def get_general_model_config(temperature, top_p, max_output_tokens): 
    return types.GenerateContentConfig(
        temperature=temperature, 
        top_p=top_p, 
        max_output_tokens=max_output_tokens
    )

def few_shot_func(model_input, prompt, config): 
    few_shot_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=config,
        contents=[prompt, model_input]
    )
    str_response = few_shot_response.text
    return str_response

model_config = get_general_model_config(0.1, 1, 500)

few_shot_prompt = """Parse JSON into a string request and only return output: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
"30-year old female patient from zipcode 78664 currently experiencing symptoms of cough, fever, and sore throat."

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"symptoms": ["Joint pain", "Loss of smell"]
}
"25-year old non-binary patient from zipcode 77055 currently experiencing symptoms of joint pain and loss of smell."

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
"15-year old male patient from zipcode 78681 currently experiencing symptoms of fatigue, loss of vision, headache, and joint pain."
"""

patient_medical_request = few_shot_func(patient_json_form, few_shot_prompt, model_config)
print(patient_medical_request)

"32-year old female patient from zipcode 70112 currently experiencing symptoms of joint pain, runny nose, loss of taste, and sore throat."



In [11]:
def grounding_func(input_prompt): 
    config_with_search = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )
    response = client.models.generate_content(
        model='gemini-2.0-flash', 
        contents=input_prompt, 
        config=config_with_search,
    )
    rc = response.candidates[0]
    return rc.content.parts[0].text

prompt = patient_medical_request + "What disease are they most likely to experience now? Only return the singular most likely disease, please."

prompt_result = grounding_func(prompt)

Markdown(prompt_result)

Given the patient's symptoms (joint pain, runny nose, loss of taste, and sore throat), the most likely disease they are currently experiencing is a **COVID-19** infection.

While other conditions like the flu or a common cold could present with some of these symptoms, the combination of loss of taste with the other symptoms makes COVID-19 the most probable diagnosis.


In [12]:
prompt = prompt_result + 'Can you get me the nearest best-rated medical facilities with their addresses that can treat the disease for zip code 70112?'

Markdown(grounding_func(prompt))

Based on the search results, here are some of the nearest and well-regarded medical facilities that may be able to assist with COVID-19 treatment in the 70112 zip code area:

*   **Concentra Downtown New Orleans:** Located at 318 Baronne Street, New Orleans, LA 70112. Concentra offers urgent care services, including treatment for illnesses like coughs, colds, and the flu, which can be relevant in addressing COVID-19 symptoms.
*   **University Medical Center New Orleans:** Situated at 2000 Canal Street, New Orleans, LA 70112. As a comprehensive medical center, it provides a wide range of services, including emergency services and affiliated clinicians who can address various health concerns.
*   **nolaHYPE:** Located at 1440 Canal St, New Orleans, LA 70112, 9th Floor (Suite 965), Tulane Tidewater Building. They offer primary care and research services.
*   **New Orleans Health Department:** Located at 1300 Perdido St. Suite 8E18 New Orleans, LA 70112. They can provide comprehensive care through healthcare facilities, nutrition programs, and preventive services.

While these facilities are in or near the specified zip code, it's advisable to check with your insurance provider and contact the facilities directly to confirm their current COVID-19 treatment protocols, availability, and specific services offered.
